# Preprocesamiento: Migraciones de Investigadores Científicos

## Objetivos

Este notebook realiza el preprocesamiento completo del dataset de migraciones de investigadores científicos, integrándolo con indicadores del Banco Mundial (WDI).

### Tareas principales:

1. **Carga y exploración inicial** del dataset de migraciones
2. **Tipado robusto** de columnas (años como Int64, booleanos, categóricas)
3. **Mapeo ISO2 → ISO3** usando `pycountry` para estandarizar códigos de países
4. **Derivación de flujos migratorios** (origen → destino) con agregaciones
5. **Integración con WDI** (población, PIB per cápita, gasto I+D)
6. **Exportación a formato optimizado** (Parquet + CSV)

## Entradas

- `../data/Scientific Researcher Migrations.csv`: Dataset principal de migraciones
- `../data/World Development Indicators/Country.csv`: Mapeo de países WDI
- `../data/World Development Indicators/Indicators.csv`: Indicadores socioeconómicos

## Salidas

- `../outputs/processed/migrations_clean.parquet|csv`: Dataset limpio de migraciones individuales
- `../outputs/processed/migration_flows.parquet|csv`: Flujos agregados origen→destino
- `../outputs/processed/country_mapping.csv`: Mapeo completo ISO2→ISO3
- `../outputs/processed/wdi_indicators.parquet|csv`: Indicadores WDI seleccionados

## 1. Importación de Librerías y Configuración

In [17]:
# Librerías estándar
import pandas as pd
import numpy as np
from pathlib import Path
import warnings

# Librería para mapeo ISO2 → ISO3
import pycountry

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✓ Librerías importadas correctamente")
print(f"  - pandas: {pd.__version__}")
print(f"  - numpy: {np.__version__}")
print(f"  - pycountry: {pycountry.__version__}")

✓ Librerías importadas correctamente
  - pandas: 2.3.3
  - numpy: 2.3.3
  - pycountry: 24.6.1


## 2. Configuración de Rutas

Establecemos las rutas relativas para trabajar de forma reproducible en cualquier entorno.

In [18]:
# Rutas base
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / 'data'
WDI_DIR = DATA_DIR / 'World Development Indicators'
OUTPUT_DIR = BASE_DIR / 'outputs' / 'processed'

# Crear directorio de salida si no existe
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("📁 Estructura de directorios:")
print(f"  - Datos entrada: {DATA_DIR}")
print(f"  - WDI: {WDI_DIR}")
print(f"  - Salida: {OUTPUT_DIR}")
print(f"\n✓ Directorio de salida {'creado' if not OUTPUT_DIR.exists() else 'verificado'}")

📁 Estructura de directorios:
  - Datos entrada: c:\Users\José Luis\Documents\GitHub\Scientific-Researcher-Migrations\data
  - WDI: c:\Users\José Luis\Documents\GitHub\Scientific-Researcher-Migrations\data\World Development Indicators
  - Salida: c:\Users\José Luis\Documents\GitHub\Scientific-Researcher-Migrations\outputs\processed

✓ Directorio de salida verificado


## 3. Carga del Dataset de Migraciones

Cargamos el dataset principal y realizamos una exploración inicial para entender su estructura.

In [19]:
# Cargar dataset de migraciones
migrations_path = DATA_DIR / 'Scientific Researcher Migrations.csv'

print(f"📊 Cargando dataset de migraciones...")
print(f"   Archivo: {migrations_path.name}\n")

df = pd.read_csv(migrations_path)

print(f"✓ Dataset cargado exitosamente")
print(f"  - Filas: {len(df):,}")
print(f"  - Columnas: {len(df.columns)}")
print(f"  - Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\n📋 Columnas disponibles:")
for col in df.columns:
    print(f"   - {col}: {df[col].dtype}")

📊 Cargando dataset de migraciones...
   Archivo: Scientific Researcher Migrations.csv

✓ Dataset cargado exitosamente
  - Filas: 741,867
  - Columnas: 8
  - Tamaño en memoria: 154.49 MB

📋 Columnas disponibles:
   - orcid_id: object
   - phd_year: float64
   - country_2016: object
   - earliest_year: float64
   - earliest_country: object
   - has_phd: bool
   - phd_country: object
   - has_migrated: bool
✓ Dataset cargado exitosamente
  - Filas: 741,867
  - Columnas: 8
  - Tamaño en memoria: 154.49 MB

📋 Columnas disponibles:
   - orcid_id: object
   - phd_year: float64
   - country_2016: object
   - earliest_year: float64
   - earliest_country: object
   - has_phd: bool
   - phd_country: object
   - has_migrated: bool


In [20]:
# Vista previa de los datos
print("📋 Primeras 10 filas del dataset:\n")
display(df.head(10))

print("\n📊 Información detallada del dataset:\n")
df.info()

📋 Primeras 10 filas del dataset:



,orcid_id,phd_year,country_2016,earliest_year,earliest_country,has_phd,phd_country,has_migrated
0,0000-0001-5000-0138,NaN,CO,2014.0,CO,False,NaN,False
1,0000-0001-5000-0736,2006.0,NaN,NaN,NaN,True,PT,False
2,0000-0001-5000-1018,2015.0,US,2005.0,US,True,US,False
3,0000-0001-5000-1181,NaN,RU,1978.0,RU,False,NaN,False
4,0000-0001-5000-1923,2016.0,GB,2004.0,GB,True,GB,False
5,0000-0001-5000-223X,1998.0,GB,1989.0,GB,True,GB,True
6,0000-0001-5000-2520,NaN,NaN,NaN,NaN,False,NaN,False
7,0000-0001-5000-311X,2002.0,NaN,NaN,NaN,True,SE,False
8,0000-0001-5000-3822,2016.0,CA,1998.0,CA,True,CA,False
9,0000-0001-5000-4390,1986.0,NaN,NaN,NaN,True,IN,False



📊 Información detallada del dataset:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741867 entries, 0 to 741866
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   orcid_id          741867 non-null  object 
 1   phd_year          287295 non-null  float64
 2   country_2016      500348 non-null  object 
 3   earliest_year     603531 non-null  float64
 4   earliest_country  603495 non-null  object 
 5   has_phd           741867 non-null  bool   
 6   phd_country       315717 non-null  object 
 7   has_migrated      741867 non-null  bool   
dtypes: bool(2), float64(2), object(4)
memory usage: 35.4+ MB


## 4. Análisis de Calidad de Datos

Evaluamos la completitud de los datos antes de proceder con la limpieza.

In [21]:
# Análisis de valores nulos
print("🔍 ANÁLISIS DE VALORES NULOS\n" + "="*70)

null_analysis = pd.DataFrame({
    'Columna': df.columns,
    'Nulos': [df[col].isna().sum() for col in df.columns],
    'Porcentaje': [f"{df[col].isna().sum() / len(df) * 100:.2f}%" for col in df.columns],
    'No Nulos': [df[col].notna().sum() for col in df.columns]
})

display(null_analysis)

# Identificar columnas críticas con muchos nulos
critical_null_pct = 50
high_null_cols = null_analysis[
    null_analysis['Nulos'] / len(df) * 100 > critical_null_pct
]['Columna'].tolist()

if high_null_cols:
    print(f"\n⚠️  Columnas con >{critical_null_pct}% de nulos:")
    for col in high_null_cols:
        pct = df[col].isna().sum() / len(df) * 100
        print(f"   - {col}: {pct:.1f}%")

🔍 ANÁLISIS DE VALORES NULOS


,Columna,Nulos,Porcentaje,No Nulos
0,orcid_id,0,0.00%,741867
1,phd_year,454572,61.27%,287295
2,country_2016,241519,32.56%,500348
3,earliest_year,138336,18.65%,603531
4,earliest_country,138372,18.65%,603495
5,has_phd,0,0.00%,741867
6,phd_country,426150,57.44%,315717
7,has_migrated,0,0.00%,741867



⚠️  Columnas con >50% de nulos:
   - phd_year: 61.3%
   - phd_country: 57.4%


In [22]:
# Análisis de duplicados
print("\n🔍 ANÁLISIS DE DUPLICADOS\n" + "="*70)

n_dup_exactos = df.duplicated().sum()
print(f"\n1️⃣  Duplicados exactos (todas las columnas):")
print(f"   - Cantidad: {n_dup_exactos:,}")
print(f"   - Porcentaje: {n_dup_exactos / len(df) * 100:.2f}%")

if 'orcid_id' in df.columns:
    n_dup_orcid = df.duplicated(subset=['orcid_id']).sum()
    print(f"\n2️⃣  Duplicados por ORCID ID:")
    print(f"   - Cantidad: {n_dup_orcid:,}")
    print(f"   - Porcentaje: {n_dup_orcid / len(df) * 100:.2f}%")
    print(f"   - Investigadores únicos: {df['orcid_id'].nunique():,}")


🔍 ANÁLISIS DE DUPLICADOS

1️⃣  Duplicados exactos (todas las columnas):
   - Cantidad: 0
   - Porcentaje: 0.00%

2️⃣  Duplicados por ORCID ID:
   - Cantidad: 0
   - Porcentaje: 0.00%

1️⃣  Duplicados exactos (todas las columnas):
   - Cantidad: 0
   - Porcentaje: 0.00%

2️⃣  Duplicados por ORCID ID:
   - Cantidad: 0
   - Porcentaje: 0.00%
   - Investigadores únicos: 741,867
   - Investigadores únicos: 741,867


## 5. Tipado Robusto de Columnas

Convertimos cada columna a su tipo de dato óptimo:
- Años → **Int64** (permite NaN)
- Booleanos → **bool**
- Países → **category**
- ID → **string**

In [23]:
print("🔧 TIPADO DE COLUMNAS\n" + "="*70)

# 1. Columna ID (orcid_id) → string
if 'orcid_id' in df.columns:
    df['orcid_id'] = df['orcid_id'].astype('string')
    print("✓ orcid_id → string")

# 2. Columnas de año → Int64 (permite NaN)
year_cols = ['phd_year', 'earliest_year']
for col in year_cols:
    if col in df.columns:
        # Convertir a numérico primero (maneja errores)
        df[col] = pd.to_numeric(df[col], errors='coerce')
        # Convertir a Int64 (nullable integer)
        df[col] = df[col].astype('Int64')
        print(f"✓ {col} → Int64")

# 3. Columnas booleanas → bool
bool_cols = ['has_phd', 'has_migrated']
for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].fillna(False).astype('bool')
        print(f"✓ {col} → bool")

# 4. Columnas de país → category (reduce memoria)
country_cols = ['country_2016', 'earliest_country', 'phd_country']
for col in country_cols:
    if col in df.columns:
        df[col] = df[col].astype('category')
        print(f"✓ {col} → category")

print(f"\n📊 Reducción de memoria después del tipado:")
print(f"   {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

🔧 TIPADO DE COLUMNAS
✓ orcid_id → string
✓ phd_year → Int64
✓ earliest_year → Int64
✓ has_phd → bool
✓ has_migrated → bool
✓ country_2016 → category
✓ earliest_country → category
✓ phd_country → category

📊 Reducción de memoria después del tipado:
   66.56 MB
✓ earliest_year → Int64
✓ has_phd → bool
✓ has_migrated → bool
✓ country_2016 → category
✓ earliest_country → category
✓ phd_country → category

📊 Reducción de memoria después del tipado:
   66.56 MB


## 6. Normalización de Nombres de Columnas

Renombramos las columnas a nombres más claros y consistentes con convenciones de nomenclatura.

In [24]:
# Mapeo de nombres de columnas
column_mapping = {
    'orcid_id': 'researcher_id',
    'phd_year': 'phd_year',
    'country_2016': 'destination',  # País actual (2016)
    'earliest_year': 'origin_year',  # Año de primera afiliación
    'earliest_country': 'origin',    # País de primera afiliación
    'has_phd': 'has_phd',
    'phd_country': 'phd_location',   # País donde obtuvo el doctorado
    'has_migrated': 'has_migrated'
}

df = df.rename(columns=column_mapping)

print("✓ Columnas renombradas:")
for old, new in column_mapping.items():
    if old != new:
        print(f"   {old} → {new}")

print(f"\n📋 Columnas finales: {list(df.columns)}")

✓ Columnas renombradas:
   orcid_id → researcher_id
   country_2016 → destination
   earliest_year → origin_year
   earliest_country → origin
   phd_country → phd_location

📋 Columnas finales: ['researcher_id', 'phd_year', 'destination', 'origin_year', 'origin', 'has_phd', 'phd_location', 'has_migrated']


## 7. Mapeo ISO2 → ISO3 con pycountry

Convertimos los códigos ISO2 (2 letras) a ISO3 (3 letras) para estandarizar y facilitar la integración con otras fuentes de datos.

In [25]:
def iso2_to_iso3(iso2_code):
    """
    Convierte código ISO2 (2 letras) a ISO3 (3 letras) usando pycountry.
    
    Args:
        iso2_code: Código ISO2 del país (ej: 'US', 'GB')
    
    Returns:
        str: Código ISO3 (ej: 'USA', 'GBR') o None si no se encuentra
    """
    if pd.isna(iso2_code) or iso2_code == '':
        return None
    
    try:
        country = pycountry.countries.get(alpha_2=str(iso2_code).upper())
        return country.alpha_3 if country else None
    except (KeyError, AttributeError):
        return None

print("🗺️  MAPEO ISO2 → ISO3\n" + "="*70)

# Crear columnas ISO3 para cada columna de país
country_cols_map = {
    'origin': 'origin_iso3',
    'destination': 'destination_iso3',
    'phd_location': 'phd_location_iso3'
}

for iso2_col, iso3_col in country_cols_map.items():
    if iso2_col in df.columns:
        df[iso3_col] = df[iso2_col].astype(str).apply(iso2_to_iso3)
        
        # Estadísticas de mapeo
        n_total = df[iso2_col].notna().sum()
        n_mapped = df[iso3_col].notna().sum()
        n_unmapped = n_total - n_mapped
        
        print(f"\n✓ {iso2_col} → {iso3_col}:")
        print(f"   - Total de códigos: {n_total:,}")
        print(f"   - Mapeados: {n_mapped:,} ({n_mapped/n_total*100:.1f}%)")
        print(f"   - No mapeados: {n_unmapped:,} ({n_unmapped/n_total*100:.1f}%)")
        
        # Mostrar códigos no mapeados si existen
        if n_unmapped > 0:
            unmapped_codes = df[
                df[iso2_col].notna() & df[iso3_col].isna()
            ][iso2_col].unique()[:10]
            print(f"   - Códigos no mapeados (muestra): {list(unmapped_codes)}")

🗺️  MAPEO ISO2 → ISO3

✓ origin → origin_iso3:
   - Total de códigos: 603,495
   - Mapeados: 603,495 (100.0%)
   - No mapeados: 0 (0.0%)

✓ origin → origin_iso3:
   - Total de códigos: 603,495
   - Mapeados: 603,495 (100.0%)
   - No mapeados: 0 (0.0%)

✓ destination → destination_iso3:
   - Total de códigos: 500,348
   - Mapeados: 500,348 (100.0%)
   - No mapeados: 0 (0.0%)

✓ destination → destination_iso3:
   - Total de códigos: 500,348
   - Mapeados: 500,348 (100.0%)
   - No mapeados: 0 (0.0%)

✓ phd_location → phd_location_iso3:
   - Total de códigos: 315,717
   - Mapeados: 315,717 (100.0%)
   - No mapeados: 0 (0.0%)

✓ phd_location → phd_location_iso3:
   - Total de códigos: 315,717
   - Mapeados: 315,717 (100.0%)
   - No mapeados: 0 (0.0%)


In [26]:
# Crear tabla de mapeo completa para referencia
print("\n📋 Creando tabla de mapeo ISO2 → ISO3...\n")

# Extraer todos los códigos ISO2 únicos
all_iso2_codes = set()
for col in ['origin', 'destination', 'phd_location']:
    if col in df.columns:
        all_iso2_codes.update(df[col].dropna().unique())

# Crear DataFrame de mapeo
mapping_df = pd.DataFrame({
    'iso2': sorted(all_iso2_codes),
})

mapping_df['iso3'] = mapping_df['iso2'].apply(iso2_to_iso3)

# Agregar nombres de países
def get_country_name(iso2_code):
    try:
        country = pycountry.countries.get(alpha_2=str(iso2_code).upper())
        return country.name if country else None
    except:
        return None

mapping_df['country_name'] = mapping_df['iso2'].apply(get_country_name)

# Ordenar y mostrar
mapping_df = mapping_df.sort_values('iso2').reset_index(drop=True)

print(f"✓ Tabla de mapeo creada: {len(mapping_df)} países")
print(f"\n📋 Primeros 20 registros del mapeo:\n")
display(mapping_df.head(20))

# Exportar mapeo
mapping_output = OUTPUT_DIR / 'country_mapping.csv'
mapping_df.to_csv(mapping_output, index=False, encoding='utf-8-sig')
print(f"\n✓ Mapeo exportado: {mapping_output}")


📋 Creando tabla de mapeo ISO2 → ISO3...

✓ Tabla de mapeo creada: 231 países

📋 Primeros 20 registros del mapeo:



,iso2,iso3,country_name
0,AD,AND,Andorra
1,AE,ARE,United Arab Emirates
2,AF,AFG,Afghanistan
3,AG,ATG,Antigua and Barbuda
4,AI,AIA,Anguilla
5,AL,ALB,Albania
6,AM,ARM,Armenia
7,AO,AGO,Angola
8,AQ,ATA,Antarctica
9,AR,ARG,Argentina



✓ Mapeo exportado: c:\Users\José Luis\Documents\GitHub\Scientific-Researcher-Migrations\outputs\processed\country_mapping.csv


## 8. Filtrado de Datos: Conservar Solo Registros con Origen o Destino

**Importante**: Mantenemos TODOS los registros que tengan al menos origen O destino definido. Solo eliminamos registros donde ambos sean nulos (no aportan información sobre flujos).

In [27]:
print("🔍 FILTRADO DE REGISTROS\n" + "="*70)

n_inicial = len(df)
print(f"\n📊 Registros iniciales: {n_inicial:,}")

# Estadísticas ANTES del filtro
print(f"\nAntes del filtro:")
print(f"  - Con origen: {df['origin'].notna().sum():,}")
print(f"  - Con destino: {df['destination'].notna().sum():,}")
print(f"  - Con origen Y destino: {(df['origin'].notna() & df['destination'].notna()).sum():,}")
print(f"  - Sin origen NI destino: {(df['origin'].isna() & df['destination'].isna()).sum():,}")

# FILTRO: Mantener registros con origen O destino (al menos uno)
df_filtered = df[
    df['origin'].notna() | df['destination'].notna()
].copy()

n_final = len(df_filtered)
n_eliminados = n_inicial - n_final

print(f"\n✓ Filtrado completado:")
print(f"  - Registros finales: {n_final:,}")
print(f"  - Registros eliminados: {n_eliminados:,} ({n_eliminados/n_inicial*100:.2f}%)")
print(f"  - Registros conservados: {n_final/n_inicial*100:.2f}%")

# Actualizar referencia
df = df_filtered

🔍 FILTRADO DE REGISTROS

📊 Registros iniciales: 741,867

Antes del filtro:
  - Con origen: 603,495
  - Con destino: 500,348
  - Con origen Y destino: 499,670
  - Sin origen NI destino: 137,694

✓ Filtrado completado:
  - Registros finales: 604,173
  - Registros eliminados: 137,694 (18.56%)
  - Registros conservados: 81.44%

✓ Filtrado completado:
  - Registros finales: 604,173
  - Registros eliminados: 137,694 (18.56%)
  - Registros conservados: 81.44%


## 9. Estadísticas Descriptivas del Dataset Limpio

In [28]:
print("📊 ESTADÍSTICAS DESCRIPTIVAS\n" + "="*70)

# Estadísticas de migraciones
print(f"\n1️⃣  MIGRACIONES:")
n_has_migrated = df['has_migrated'].sum()
print(f"   - Total de investigadores que migraron: {n_has_migrated:,} ({n_has_migrated/len(df)*100:.1f}%)")
print(f"   - Total sin migración: {len(df) - n_has_migrated:,} ({(len(df)-n_has_migrated)/len(df)*100:.1f}%)")

# Estadísticas de doctorados
print(f"\n2️⃣  DOCTORADOS:")
n_has_phd = df['has_phd'].sum()
print(f"   - Investigadores con PhD: {n_has_phd:,} ({n_has_phd/len(df)*100:.1f}%)")

# Top países de origen
print(f"\n3️⃣  TOP 10 PAÍSES DE ORIGEN:")
top_origin = df['origin'].value_counts().head(10)
for i, (country, count) in enumerate(top_origin.items(), 1):
    print(f"   {i:2d}. {country}: {count:,} ({count/df['origin'].notna().sum()*100:.1f}%)")

# Top países de destino
print(f"\n4️⃣  TOP 10 PAÍSES DE DESTINO (2016):")
top_dest = df['destination'].value_counts().head(10)
for i, (country, count) in enumerate(top_dest.items(), 1):
    print(f"   {i:2d}. {country}: {count:,} ({count/df['destination'].notna().sum()*100:.1f}%)")

# Distribución temporal
print(f"\n5️⃣  DISTRIBUCIÓN TEMPORAL:")
if df['phd_year'].notna().sum() > 0:
    print(f"   PhD years:")
    print(f"   - Rango: {df['phd_year'].min()} - {df['phd_year'].max()}")
    print(f"   - Media: {df['phd_year'].mean():.0f}")
    print(f"   - Mediana: {df['phd_year'].median():.0f}")

if df['origin_year'].notna().sum() > 0:
    print(f"   Origin years:")
    print(f"   - Rango: {df['origin_year'].min()} - {df['origin_year'].max()}")
    print(f"   - Media: {df['origin_year'].mean():.0f}")
    print(f"   - Mediana: {df['origin_year'].median():.0f}")

📊 ESTADÍSTICAS DESCRIPTIVAS

1️⃣  MIGRACIONES:
   - Total de investigadores que migraron: 107,921 (17.9%)
   - Total sin migración: 496,252 (82.1%)

2️⃣  DOCTORADOS:
   - Investigadores con PhD: 298,702 (49.4%)

3️⃣  TOP 10 PAÍSES DE ORIGEN:
    1. US: 96,706 (16.0%)
    2. IN: 40,124 (6.6%)
    3. BR: 39,902 (6.6%)
    4. CN: 38,042 (6.3%)
    5. GB: 37,977 (6.3%)
    6. ES: 30,502 (5.1%)
    7. IT: 25,146 (4.2%)
    8. RU: 19,120 (3.2%)
    9. PT: 17,538 (2.9%)
   10. AU: 15,499 (2.6%)

4️⃣  TOP 10 PAÍSES DE DESTINO (2016):
    1. US: 88,930 (17.8%)
    2. BR: 32,731 (6.5%)
    3. GB: 32,425 (6.5%)
    4. IN: 27,256 (5.4%)
    5. CN: 25,585 (5.1%)
    6. ES: 25,160 (5.0%)
    7. IT: 19,647 (3.9%)
    8. AU: 16,550 (3.3%)
    9. RU: 14,424 (2.9%)
   10. PT: 14,111 (2.8%)

5️⃣  DISTRIBUCIÓN TEMPORAL:
   PhD years:
   - Rango: 1947 - 2017
   - Media: 2008
   - Mediana: 2012
   Origin years:
   - Rango: 1913 - 2017
   - Media: 2000
   - Mediana: 2003


## 10. Derivación de Flujos Migratorios (Origen → Destino)

Creamos una tabla agregada de flujos migratorios entre países, contando el número de investigadores que se movieron de cada país origen a cada país destino.

In [29]:
print("🌍 DERIVACIÓN DE FLUJOS MIGRATORIOS\n" + "="*70)

# Filtrar solo investigadores que han migrado con origen Y destino definidos
# Convertir categorías a string para permitir comparación
df_flows = df[
    (df['has_migrated'] == True) &
    (df['origin'].notna()) &
    (df['destination'].notna()) &
    (df['origin'].astype(str) != df['destination'].astype(str))  # Excluir "migraciones" dentro del mismo país
].copy()

print(f"✓ Registros válidos para flujos: {len(df_flows):,}")
print(f"  ({len(df_flows)/len(df)*100:.1f}% del dataset total)\n")

# Agregación por origen → destino usando Named Aggregations (evita MultiIndex de columnas)
flows_aggregated = df_flows.groupby(
    ['origin', 'destination', 'origin_iso3', 'destination_iso3'],
    observed=True  # Solo categorías observadas
).agg(
    n_researchers=('researcher_id', 'count'),
    phd_year_min=('phd_year', 'min'),
    phd_year_max=('phd_year', 'max'),
    phd_year_mean=('phd_year', 'mean'),
    origin_year_min=('origin_year', 'min'),
    origin_year_max=('origin_year', 'max'),
    origin_year_mean=('origin_year', 'mean'),
).reset_index()

# Crear etiqueta de ruta
flows_aggregated['route'] = (
    flows_aggregated['origin'].astype(str) + ' → ' +
    flows_aggregated['destination'].astype(str)
)

# Ordenar por número de investigadores (descendente)
flows_aggregated = flows_aggregated.sort_values(
    'n_researchers',
    ascending=False
).reset_index(drop=True)

# Tipar columnas de año como Int64 (seguro ante NaN) y consolidar tipo entero para n_researchers
year_cols_flows = [
    'phd_year_min', 'phd_year_max', 'phd_year_mean',
    'origin_year_min', 'origin_year_max', 'origin_year_mean'
]
for col in year_cols_flows:
    flows_aggregated[col] = pd.to_numeric(flows_aggregated[col], errors='coerce')
    flows_aggregated[col] = flows_aggregated[col].round(0).astype('Int64')

flows_aggregated['n_researchers'] = pd.to_numeric(flows_aggregated['n_researchers'], errors='coerce').astype('Int64')

print(f"✓ Flujos agregados:")
print(f"  - Total de rutas únicas: {len(flows_aggregated):,}")
print(f"  - Países origen únicos: {flows_aggregated['origin'].nunique()}")
print(f"  - Países destino únicos: {flows_aggregated['destination'].nunique()}")
print(f"\n📊 Top 20 rutas migratorias:\n")
display(flows_aggregated[[
    'route', 'n_researchers', 'phd_year_mean', 'origin_year_mean'
]].head(20))


🌍 DERIVACIÓN DE FLUJOS MIGRATORIOS
✓ Registros válidos para flujos: 62,004
  (10.3% del dataset total)

✓ Flujos agregados:
  - Total de rutas únicas: 4,249
  - Países origen únicos: 194
  - Países destino únicos: 202

📊 Top 20 rutas migratorias:

✓ Registros válidos para flujos: 62,004
  (10.3% del dataset total)

✓ Flujos agregados:
  - Total de rutas únicas: 4,249
  - Países origen únicos: 194
  - Países destino únicos: 202

📊 Top 20 rutas migratorias:



,route,n_researchers,phd_year_mean,origin_year_mean
0,CN → US,3508,2011,2000
1,IN → US,1916,2009,1998
2,CA → US,1029,2004,1995
3,GB → US,940,2000,1992
4,US → GB,786,2006,1997
5,GB → AU,719,2000,1991
6,US → KR,709,2000,1995
7,US → CA,616,2001,1993
8,US → TW,610,1997,1992
9,KR → US,562,2010,1997


## 11. Integración con World Development Indicators (WDI)

Cargamos indicadores socioeconómicos del Banco Mundial para enriquecer el análisis:
- **SP.POP.TOTL**: Población total
- **NY.GDP.PCAP.CD**: PIB per cápita (USD corrientes)
- **GB.XPD.RSDV.GD.ZS**: Gasto en I+D (% del PIB)
- **SP.POP.SCIE.RD.P6**: Investigadores en I+D (por millón hab.)

In [30]:
# Verificar disponibilidad de archivos WDI
wdi_country_path = WDI_DIR / 'Country.csv'
wdi_indicators_path = WDI_DIR / 'Indicators.csv'

print("🌐 INTEGRACIÓN CON WORLD DEVELOPMENT INDICATORS\n" + "="*70)
print(f"\nVerificando archivos WDI:")
print(f"  - Country.csv: {'✓' if wdi_country_path.exists() else '✗'}")
print(f"  - Indicators.csv: {'✓' if wdi_indicators_path.exists() else '✗'}")

WDI_AVAILABLE = wdi_country_path.exists() and wdi_indicators_path.exists()

if not WDI_AVAILABLE:
    print(f"\n⚠️  Archivos WDI no disponibles. Saltando integración.")
    print(f"   El análisis continuará sin indicadores socioeconómicos.")

🌐 INTEGRACIÓN CON WORLD DEVELOPMENT INDICATORS

Verificando archivos WDI:
  - Country.csv: ✓
  - Indicators.csv: ✓


In [31]:
if WDI_AVAILABLE:
    # Cargar metadatos de países
    print(f"\n📊 Cargando WDI Country.csv...")
    wdi_country = pd.read_csv(wdi_country_path, encoding='utf-8')
    
    print(f"✓ Metadatos de países cargados: {len(wdi_country)} registros")
    print(f"  Columnas clave: {[col for col in wdi_country.columns if 'Code' in col or 'Name' in col]}")
    
    # Crear mapeo ISO2 → ISO3 desde WDI
    wdi_mapping = wdi_country[[
        'Alpha2Code', 'CountryCode', 'ShortName', 'TableName'
    ]].copy()
    wdi_mapping.columns = ['iso2_wdi', 'iso3_wdi', 'country_short', 'country_full']
    
    print(f"\n✓ Mapeo WDI creado: {len(wdi_mapping)} países")
    display(wdi_mapping.head(10))


📊 Cargando WDI Country.csv...
✓ Metadatos de países cargados: 247 registros
  Columnas clave: ['CountryCode', 'ShortName', 'TableName', 'LongName', 'Alpha2Code', 'Wb2Code']

✓ Mapeo WDI creado: 247 países


,iso2_wdi,iso3_wdi,country_short,country_full
0,AF,AFG,Afghanistan,Afghanistan
1,AL,ALB,Albania,Albania
2,DZ,DZA,Algeria,Algeria
3,AS,ASM,American Samoa,American Samoa
4,AD,ADO,Andorra,Andorra
5,AO,AGO,Angola,Angola
6,AG,ATG,Antigua and Barbuda,Antigua and Barbuda
7,1A,ARB,Arab World,Arab World
8,AR,ARG,Argentina,Argentina
9,AM,ARM,Armenia,Armenia


In [32]:
if WDI_AVAILABLE:
    # Indicadores de interés
    WDI_INDICATORS = {
        'SP.POP.TOTL': 'Población total',
        'NY.GDP.PCAP.CD': 'PIB per cápita (USD)',
        'GB.XPD.RSDV.GD.ZS': 'Gasto I+D (% PIB)',
        'SP.POP.SCIE.RD.P6': 'Investigadores por millón hab.'
    }
    
    print(f"\n📊 Cargando indicadores WDI...")
    print(f"   Indicadores objetivo:")
    for code, desc in WDI_INDICATORS.items():
        print(f"   - {code}: {desc}")
    
    # Cargar Indicators.csv (advertencia: archivo muy grande)
    print(f"\n⏳ Cargando Indicators.csv (esto puede tardar varios minutos)...")
    
    # Cargar solo las columnas necesarias y filtrar por indicadores
    wdi_indicators = pd.read_csv(
        wdi_indicators_path,
        usecols=['CountryCode', 'IndicatorCode', 'Year', 'Value'],
        dtype={'Year': 'Int64', 'Value': 'float64'}
    )
    
    # Filtrar solo indicadores de interés
    wdi_indicators = wdi_indicators[
        wdi_indicators['IndicatorCode'].isin(WDI_INDICATORS.keys())
    ].copy()
    
    print(f"\n✓ Indicadores WDI cargados:")
    print(f"  - Total de registros: {len(wdi_indicators):,}")
    print(f"  - Países únicos: {wdi_indicators['CountryCode'].nunique()}")
    print(f"  - Indicadores únicos: {wdi_indicators['IndicatorCode'].nunique()}")
    print(f"  - Rango de años: {wdi_indicators['Year'].min()} - {wdi_indicators['Year'].max()}")
    
    # Agregar nombre descriptivo del indicador
    wdi_indicators['IndicatorName'] = wdi_indicators['IndicatorCode'].map(WDI_INDICATORS)
    
    # Renombrar CountryCode a iso3 para consistencia
    wdi_indicators = wdi_indicators.rename(columns={'CountryCode': 'iso3'})
    
    print(f"\n📋 Registros por indicador:")
    indicator_counts = wdi_indicators.groupby(['IndicatorCode', 'IndicatorName']).size()
    for (code, name), count in indicator_counts.items():
        print(f"   - {name}: {count:,} registros")
    
    print(f"\n📋 Vista previa de indicadores WDI:\n")
    display(wdi_indicators.head(20))


📊 Cargando indicadores WDI...
   Indicadores objetivo:
   - SP.POP.TOTL: Población total
   - NY.GDP.PCAP.CD: PIB per cápita (USD)
   - GB.XPD.RSDV.GD.ZS: Gasto I+D (% PIB)
   - SP.POP.SCIE.RD.P6: Investigadores por millón hab.

⏳ Cargando Indicators.csv (esto puede tardar varios minutos)...

✓ Indicadores WDI cargados:
  - Total de registros: 26,942
  - Países únicos: 247
  - Indicadores únicos: 4
  - Rango de años: 1960 - 2014

📋 Registros por indicador:
   - Gasto I+D (% PIB): 1,744 registros
   - PIB per cápita (USD): 10,343 registros
   - Investigadores por millón hab.: 1,371 registros
   - Población total: 13,484 registros

📋 Vista previa de indicadores WDI:


✓ Indicadores WDI cargados:
  - Total de registros: 26,942
  - Países únicos: 247
  - Indicadores únicos: 4
  - Rango de años: 1960 - 2014

📋 Registros por indicador:
   - Gasto I+D (% PIB): 1,744 registros
   - PIB per cápita (USD): 10,343 registros
   - Investigadores por millón hab.: 1,371 registros
   - Población total

,iso3,IndicatorCode,Year,Value,IndicatorName
73,ARB,SP.POP.TOTL,1960,9.249590e+07,Población total
95,CSS,NY.GDP.PCAP.CD,1960,4.574647e+02,PIB per cápita (USD)
150,CSS,SP.POP.TOTL,1960,4.190810e+06,Población total
221,CEB,SP.POP.TOTL,1960,9.140158e+07,Población total
264,EAS,NY.GDP.PCAP.CD,1960,1.468141e+02,PIB per cápita (USD)
341,EAS,SP.POP.TOTL,1960,1.042475e+09,Población total
377,EAP,NY.GDP.PCAP.CD,1960,8.931964e+01,PIB per cápita (USD)
462,EAP,SP.POP.TOTL,1960,8.964930e+08,Población total
518,EMU,NY.GDP.PCAP.CD,1960,9.245714e+02,PIB per cápita (USD)
581,EMU,SP.POP.TOTL,1960,2.653965e+08,Población total


## 12. Exportación de Datasets Procesados

Exportamos todos los datasets limpios a formatos optimizados:
- **Parquet**: Formato columnar comprimido (si pyarrow disponible)
- **CSV**: Formato universal de respaldo

In [33]:
def export_dataset(df, base_name, description):
    """
    Exporta un DataFrame a CSV (siempre) y Parquet (si disponible).
    
    Args:
        df: DataFrame a exportar
        base_name: Nombre base del archivo (sin extensión)
        description: Descripción del dataset para logging
    """
    csv_path = OUTPUT_DIR / f"{base_name}.csv"
    parquet_path = OUTPUT_DIR / f"{base_name}.parquet"
    
    print(f"\n📦 Exportando: {description}")
    print(f"   Registros: {len(df):,} | Columnas: {len(df.columns)}")
    
    # 1. Exportar CSV (siempre)
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    csv_size = csv_path.stat().st_size / 1024**2
    print(f"   ✓ CSV: {csv_path.name} ({csv_size:.2f} MB)")
    
    # 2. Intentar exportar Parquet
    try:
        df.to_parquet(parquet_path, engine='pyarrow', compression='snappy', index=False)
        parquet_size = parquet_path.stat().st_size / 1024**2
        compression_pct = (1 - parquet_size/csv_size) * 100
        print(f"   ✓ Parquet: {parquet_path.name} ({parquet_size:.2f} MB, {compression_pct:.1f}% compresión)")
    except ImportError:
        print(f"   ⚠️  Parquet no exportado (pyarrow no disponible)")
    except Exception as e:
        print(f"   ⚠️  Error al exportar Parquet: {e}")

print("💾 EXPORTACIÓN DE DATASETS\n" + "="*70)

💾 EXPORTACIÓN DE DATASETS


In [34]:
# 1. Dataset de migraciones individuales (limpio)
export_dataset(
    df,
    'migrations_clean',
    'Dataset de migraciones individuales (limpio)'
)


📦 Exportando: Dataset de migraciones individuales (limpio)
   Registros: 604,173 | Columnas: 11
   ✓ CSV: migrations_clean.csv (33.24 MB)
   ✓ CSV: migrations_clean.csv (33.24 MB)
   ✓ Parquet: migrations_clean.parquet (7.83 MB, 76.5% compresión)
   ✓ Parquet: migrations_clean.parquet (7.83 MB, 76.5% compresión)


In [35]:
# 2. Flujos migratorios agregados
export_dataset(
    flows_aggregated,
    'migration_flows',
    'Flujos migratorios agregados (origen → destino)'
)


📦 Exportando: Flujos migratorios agregados (origen → destino)
   Registros: 4,249 | Columnas: 12
   ✓ CSV: migration_flows.csv (0.22 MB)
   ✓ Parquet: migration_flows.parquet (0.07 MB, 67.5% compresión)


In [36]:
# 3. Indicadores WDI (si disponibles)
if WDI_AVAILABLE:
    export_dataset(
        wdi_indicators,
        'wdi_indicators',
        'Indicadores World Development (WDI)'
    )


📦 Exportando: Indicadores World Development (WDI)
   Registros: 26,942 | Columnas: 5
   ✓ CSV: wdi_indicators.csv (1.45 MB)
   ✓ Parquet: wdi_indicators.parquet (0.25 MB, 83.0% compresión)


## 13. Resumen Final y Próximos Pasos

In [37]:
print("="*70)
print("✅ PREPROCESAMIENTO COMPLETADO EXITOSAMENTE")
print("="*70)

print(f"\n📊 RESUMEN DE DATASETS GENERADOS:\n")

output_files = [
    ('migrations_clean', 'Dataset de migraciones individuales (limpio)'),
    ('migration_flows', 'Flujos migratorios agregados (origen → destino)'),
    ('country_mapping', 'Mapeo ISO2 → ISO3 de países'),
    ('wdi_indicators', 'Indicadores World Development (WDI)') if WDI_AVAILABLE else None
]

for i, item in enumerate([x for x in output_files if x], 1):
    base_name, desc = item
    
    # Verificar archivos existentes
    csv_path = OUTPUT_DIR / f"{base_name}.csv"
    parquet_path = OUTPUT_DIR / f"{base_name}.parquet"
    
    print(f"{i}. {desc}")
    if csv_path.exists():
        csv_size = csv_path.stat().st_size / 1024**2
        print(f"   ✓ {csv_path.name} ({csv_size:.2f} MB)")
    if parquet_path.exists():
        parquet_size = parquet_path.stat().st_size / 1024**2
        print(f"   ✓ {parquet_path.name} ({parquet_size:.2f} MB)")
    print()

print(f"📁 Todos los archivos guardados en: {OUTPUT_DIR}")

print(f"\n🎯 PRÓXIMOS PASOS:\n")
print(f"   1. Análisis Exploratorio de Datos (EDA):")
print(f"      - Tendencias temporales de migraciones")
print(f"      - Brain gain/brain drain por país")
print(f"      - Top rutas migratorias")
print(f"      - Análisis per cápita con WDI")
print(f"\n   2. Visualizaciones:")
print(f"      - Mapas de flujos migratorios (chord diagram)")
print(f"      - Series temporales")
print(f"      - Rankings interactivos")
print(f"\n   3. Dashboard interactivo con Streamlit o Dash")

print(f"\n{'='*70}")
print(f"✨ Preprocesamiento finalizado sin warnings")
print(f"{'='*70}")

✅ PREPROCESAMIENTO COMPLETADO EXITOSAMENTE

📊 RESUMEN DE DATASETS GENERADOS:

1. Dataset de migraciones individuales (limpio)
   ✓ migrations_clean.csv (33.24 MB)
   ✓ migrations_clean.parquet (7.83 MB)

2. Flujos migratorios agregados (origen → destino)
   ✓ migration_flows.csv (0.22 MB)
   ✓ migration_flows.parquet (0.07 MB)

3. Mapeo ISO2 → ISO3 de países
   ✓ country_mapping.csv (0.00 MB)

4. Indicadores World Development (WDI)
   ✓ wdi_indicators.csv (1.45 MB)
   ✓ wdi_indicators.parquet (0.25 MB)

📁 Todos los archivos guardados en: c:\Users\José Luis\Documents\GitHub\Scientific-Researcher-Migrations\outputs\processed

🎯 PRÓXIMOS PASOS:

   1. Análisis Exploratorio de Datos (EDA):
      - Tendencias temporales de migraciones
      - Brain gain/brain drain por país
      - Top rutas migratorias
      - Análisis per cápita con WDI

   2. Visualizaciones:
      - Mapas de flujos migratorios (chord diagram)
      - Series temporales
      - Rankings interactivos

   3. Dashboard inter